In [ ]:
import sys
sys.path.insert(0, '/Users/sophied/Library/CloudStorage/OneDrive-Personal/NUS/Year 4 Sem 2/DSA4263/DSA4263_VOC/src')

In [ ]:
#ssh -NL 888:127.0.0.1:8888 <username>@<IP-address> -i <key-file-location>

In [47]:
from transformations import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
from transformers import pipeline, BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments
import torch
from ray import tune
import ray

import pandas as pd
import numpy as np
import re

In [44]:
seed = 4263

In [ ]:
!ray start

In [45]:
# Load the dataset
df = pd.read_csv('../data/reviews.csv', encoding= 'unicode_escape')
df.head()

,Sentiment,Time,Text
0,positive,18/6/21,This is a very healthy dog food. Good for thei...
1,positive,7/7/21,I've been very pleased with the Natural Balanc...
2,positive,18/6/21,"Before I was educated about feline nutrition, ..."
3,positive,7/7/21,"My holistic vet recommended this, along with a..."
4,positive,1/7/21,I bought this coffee because its much cheaper ...


In [49]:
df['clean_text'] = df['Text'].apply(lambda x: get_cleantext(x))
df['label'] = df.Sentiment.map({"positive": 1, "negative": 0})

In [ ]:
df.head()

In [50]:
df_clean = df[['clean_text', 'label']]
X = list(df_clean['clean_text'])
y = list(df_clean['label'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y, random_state = 4263)

### Training

In [51]:
# Define pretrained tokenizer and model
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [52]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [53]:
X_train_tokenized = tokenizer(X_train, padding = True, truncation = True, max_length = 512)
X_test_tokenized = tokenizer(X_test, padding = True, truncation = True, max_length = 512)

In [54]:
train_dataset = Dataset(X_train_tokenized, y_train)
test_dataset = Dataset(X_test_tokenized, y_test)

In [55]:
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    #recall = recall_score(y_true=labels, y_pred=pred)
    #precision = precision_score(y_true=labels, y_pred=pred)
    #auc = auc_roc_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy}

In [ ]:
def ray_hp_space(trial):
    return {
        'learning_rate': tune.choice([2e-5, 3e-5, 5e-5]),
        'num_train_epochs': tune.choice([3, 5])
    }

In [ ]:
def model_init(trial):
    return BertForSequenceClassification.from_pretrained(model_name)

In [ ]:
trainer = Trainer(
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    compute_metrics = compute_metrics,
    model_init = model_init
)

In [ ]:
# Perform hyperparameter tuning
best_trial = trainer.hyperparameter_search(
    direction = "maximize",
    backend = "ray",
    hp_space = ray_hp_space,
    n_trials = 2
)

In [ ]:
print(best_trial.hyperparameters)

### Test

In [ ]:
#final_model = trainer.get_best_model

In [ ]:
# Load the sentiment analysis pipeline using BERT with the best hyperparameters
""" sentiment_analysis = pipeline(
    "sentiment-analysis", 
    model = final_model, 
    device = 0,
    tokenizer = "bert-base-uncased", 
    framework = "pt") """

In [ ]:
# Define a function to apply sentiment analysis on each review in the dataset
""" def get_sentiment(review):
    result = sentiment_analysis(review)
    return result[0]['label']

# Apply sentiment analysis on each review in the dataset
df_reviews['sentiment'] = df_reviews['review'].apply(get_sentiment)

# Print the sentiment label and score for each review
df_reviews[['review', 'sentiment']] """

### Predict using Base Model

In [ ]:
sentiment_analysis = pipeline(
    "sentiment-analysis", 
    model = model, 
    tokenizer = tokenizer, 
    framework = "pt", 
    truncation = True, 
    max_length = 512)

In [ ]:
def get_sentiment(review):
    result = sentiment_analysis(review)
    return result[0]['label']

# Apply sentiment analysis on each review in the dataset
df['pred_sentiment'] = df['clean_text'].apply(get_sentiment)

# Print the sentiment label and score for each review
df[['clean_text', 'pred_sentiment']]

In [ ]:
df.to_csv('results.csv')

## Train and Predict